In [1]:
import numpy as np
import pandas as pd

In [4]:
all_tickers = pd.read_excel('mapping.xlsx')[['Ticker', 'Market']]

In [5]:
all_tickers

,Ticker,Market
0,ARGT,Emerging
1,EWA,Developed
2,EWO,Developed
3,EWK,Developed
4,EWZ,Emerging
5,EWC,Developed
6,ECH,Emerging
7,MCHI,Emerging
8,ICOL,Emerging
9,EDEN,Developed
